## Tensorflow 2.x Setup


In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

import random
import numpy as np
import math
import matplotlib.pyplot as plt

# tensorflow
import tensorflow as tf
print(tf.__version__)

# pytorch
import torch
from torch import nn
print(torch.__version__)

# scikit-learn
import sklearn
from sklearn import cluster, decomposition, manifold, metrics
import pandas as pd

import scipy
from scipy.stats import entropy
from scipy.stats import dirichlet

# access google file system
from google.colab import drive
drive.mount('/content/drive')
cwd = '/content/drive/My Drive/moncrief_2021/'

Found GPU at: /device:GPU:0
2.7.0
1.10.0+cu111
Mounted at /content/drive


### Get action probabilities

In [ ]:
# Example:
# shadow (seed 0~4)
# IN: label_0 ... label_4
# OUT: label_5 ... label_9

# attack
# concat(shadow_seed_0, label_0) = IN
# concat(shadow_seed_5, label_5) = OUT

# attack on concat(shadow_seed_1, label_1)

# victim, whether seed 3 is used in training
# attack on concat(victim_seed_3, label_3) ==> IN/OUT

In [ ]:
df = pd.read_csv(cwd+'data/mr_label_1/probabilities.csv')
df0 = pd.read_csv(cwd+'data/mr_label_5/probabilities.csv')
df1 = pd.read_csv(cwd+'data/mr_shadow_1/probabilities.csv')
df2 = pd.read_csv(cwd+'data/mr_shadow_5/probabilities.csv')

In [ ]:
def reshape_data(dataframe):
    # reshape the data into the shape that fit the attack model
    data = dataframe.to_numpy()
    data = data.reshape((len(data)//128, 128, 7))
    data = data.swapaxes(1,2)
    return data

data0 = reshape_data(df)
data00 = reshape_data(df0)
data1 = reshape_data(df1)
data2 = reshape_data(df2)
data0.shape

(1600, 7, 128)

In [ ]:
in_data = np.concatenate([np.concatenate([data0[:1600], data1[:1600]], axis=1),
                          np.concatenate([data00[:1600], data2[:1600]], axis=1)])

out_data = np.concatenate([np.concatenate([data0[:1600], data2[:1600]], axis=1),
                           np.concatenate([data00[:1600], data1[:1600]], axis=1)])
in_data.shape, out_data.shape

((3200, 14, 128), (3200, 14, 128))

In [ ]:
def get_att_data(in_data, out_data):
    # Generate IN/OUT labels for the data
    in_label = [1.0]*len(in_data)
    out_label = [0.0]*len(out_data)
    labels = in_label + out_label
    in_data = [d for d in in_data]
    out_data = [d for d in out_data]
    data = in_data + out_data

    c = list(zip(data, labels))
    random.shuffle(c)
    data, labels = zip(*c)
    return np.array(data), np.array(labels)

def get_label_vector(labels):
    # Convert labels into vector form (one-hot embedding)
    label_vectors = np.zeros((len(labels), np.max(labels)+1))
    for i in range(len(labels)):
        label_vectors[i, labels[i]] = 1
    return label_vectors

In [ ]:
data, labels = get_att_data(in_data, out_data)
labels = get_label_vector(np.array(labels, dtype=np.int))

test_size = 2000
data, v_data = data[:-test_size], data[-test_size:]
labels, v_labels = labels[:-test_size], labels[-test_size:]

data.shape, labels.shape

((4400, 14, 128), (4400, 2))

### Attack Model

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Softmax, LSTM
def build_att():
    model = Sequential()
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(LSTM(48, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2))
    model.add(Softmax())
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    precision = keras.metrics.Precision(class_id=0)
    recall = keras.metrics.Recall(class_id=0)
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', precision, recall])
    return model

In [ ]:
attack_model = build_att()
attack_model.fit(x=data, y=labels, batch_size=64, epochs=15, verbose=True)
attack_model.evaluate(x=v_data, y=v_labels, batch_size=64)
# attack_model.save_weights(cwd+'models/mr_att_model')

Epoch 1/15
69/69 [==============================] - 4s 11ms/step - loss: 0.6952 - accuracy: 0.5064 - precision_2: 0.5074 - recall_2: 0.5102
Epoch 2/15
69/69 [==============================] - 1s 12ms/step - loss: 0.6963 - accuracy: 0.4973 - precision_2: 0.4988 - recall_2: 0.6385
Epoch 3/15
69/69 [==============================] - 1s 11ms/step - loss: 0.6938 - accuracy: 0.5109 - precision_2: 0.5090 - recall_2: 0.6812
Epoch 4/15
69/69 [==============================] - 1s 12ms/step - loss: 0.6931 - accuracy: 0.5005 - precision_2: 0.5020 - recall_2: 0.4009
Epoch 5/15
69/69 [==============================] - 1s 11ms/step - loss: 0.6873 - accuracy: 0.5611 - precision_2: 0.5492 - recall_2: 0.6934
Epoch 6/15
69/69 [==============================] - 1s 11ms/step - loss: 0.6571 - accuracy: 0.6464 - precision_2: 0.6591 - recall_2: 0.6095
Epoch 7/15
69/69 [==============================] - 1s 11ms/step - loss: 0.5159 - accuracy: 0.8093 - precision_2: 0.8153 - recall_2: 0.8009
Epoch 8/15
69/69 [==

[0.015675978735089302,
 0.9944999814033508,
 0.9949698448181152,
 0.9939698576927185]